In [ ]:
import pandas as pd
import numpy as np
from keras.datasets import boston_housing 
from keras import models, layers, optimizers

In [4]:
# Descargando datos
(train_data , train_labels) ,(test_data,test_labels) = boston_housing.load_data()

In [5]:
train_data.shape

(404, 13)

In [6]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

##### Como los datos están muy dispersos lo ideal es normalizarlos

In [9]:
# Normalización. Restamos la media y dividimos por la desviacion estándar
mean = train_data.mean(axis=0)
train_data = train_data  - mean
std = train_data.std(axis=0)
train_data = train_data / std

test_data = test_data - mean
test_data = test_data / std

#### Definiendo nuestra red

In [16]:
def build_model_regression(lr_var, input_data):
    model = models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(input_data,)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizers.RMSprop(learning_rate=lr_var), loss='mse',metrics=['mae'])
    return model

In [20]:
# K-fold validation
k = 4
num_val_samples = len(train_data) // k
num_epoch = 80
all_history = []

In [18]:
for i in range(k):
    print(f"Fold: {i}")
    val_data = train_data[i*num_val_samples: (i+1) * num_val_samples]
    val_targets = train_labels[i*num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i+1) * num_val_samples:]
        ],
        axis= 0   
    )
    
    partial_train_targets = np.concatenate(
        [train_labels[:i * num_val_samples],
         train_targets[(i+1) * num_val_samples:]
        ],
        axis= 0   
    )
    
    model = build_model_regression(0.001, 13)
    
    history = model.fit(partial_train_data, partial_train_targets,
                        epochs=num_epoch,
                        batch_size = 16, 
                        validation_data = (val_data, val_targets),
                        verbose=0)
    
    all_history.append(history.history['val_mae'])

Fold: 0
Fold: 1
Fold: 2
Fold: 3


In [19]:
# Media de todos los MAE
len(all_history[0])

80